## Imports, Setup, Helper Functions

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

THRESHOLDS = [0.25, 0.5, 0.75, 0.9]

In [2]:
def decide(p: float, t: float) -> bool:
    # decision rule: True if model answers
    # assumption: model only answers when p > t
    return p > t


def penalty_score(pred: str, gold: str, p: float, t: float) -> float:
    # our penalty adjusted scoring
    # if p <= t -> the model abstains -> score = 0.
    # if p > t and prediction == gold -> score = +1.
    # if p > t and prediction != gold -> score = - (p * t) / (1 - t)
    if p <= t:
        return 0.0
    if pred == gold:
        return 1.0
    return round(- (p * t) / (1.0 - t),2)


def accuracy_at_threshold(df: pd.DataFrame, t: float) -> float:
    # fraction of answered questions that are correct
    # acc@t = (correct with p>t) / (# answered with p>t)
    answered = df["confidence"] > t
    answered_n = answered.sum()
    if answered_n == 0:
        return 0.0
    correct = (df["predicted_answer"] == df["answer"]) & answered
    return round(100*float(correct.sum() / answered_n),2)

def coverage(df: pd.DataFrame, t: float) -> float:
    # frac of total questions the model answers
    # cov = (# p>t) / total

    if len(df) == 0:
        return 0.0
    return round(100*float((df["confidence"] > t).sum() / len(df)),2)

def penalty_adjusted_mean(df: pd.DataFrame, t: float) -> float:
    # mean penalty-adjusted score across all rows 
    # includes abstains = 0

    scores = [
        penalty_score(r.predicted_answer, r.answer, float(r.confidence), t)
        for r in df.itertuples(index=False)
    ]
    return round(float(np.mean(scores)) if scores else 0.0,2)

def overconfidence_rate(df: pd.DataFrame, t: float) -> float:
    # fraction of questions where the model is wrong but p>t
    # over = (# wrong & p>t) / total
    if len(df) == 0:
        return 0.0
    mask = (df["predicted_answer"] != df["answer"]) & (df["confidence"] > t)
    return round(100*float(mask.sum() / len(df)))


## Qwen Evaluation

In [3]:
CSV_PATH = Path("../inference/outputs/qwen-mmlu.csv")
OUTPUT_PATH = Path("outputs")

In [4]:
df = pd.read_csv(CSV_PATH)

df["answer"] = df["answer"].astype(str).str.strip().str.upper()
df["predicted_answer"] = df["predicted_answer"].astype(str).str.strip().str.upper()
df["confidence"] = round(100*pd.to_numeric(df["confidence"], errors="coerce").fillna(0.0),2)
df["threshold"] = pd.to_numeric(df["threshold"], errors="coerce")

print(f"Loaded {len(df)} rows from {CSV_PATH.name}")
print("Thresholds found:", sorted(df["threshold"].unique()))
df.head(3)

Loaded 3400 rows from qwen-mmlu.csv
Thresholds found: [np.float64(0.25), np.float64(0.5), np.float64(0.75), np.float64(0.9)]


,id,threshold,question,choices,answer,predicted_answer,confidence
0,596,0.25,A manufacturer is currently selling 2000 units...,['$2.50' '$1.90' '$2.70' '$2.60' '$1.80' '$2.2...,F,IDK,50.00
1,814,0.25,Fred Lowes is a typewriter salesman. He receiv...,['$210' '$200' '$225' '$175' '$195' '$150' '$2...,I,B,83.33
2,817,0.25,"Mary Redmond purchased a $28,500 home with 20%...",['$305' '$190' '$171' '$285.50' '$399' '$323' ...,F,IDK,100.00


In [5]:
dfs_by_t = {t: df[df["threshold"] == t].copy() for t in THRESHOLDS}

for t in THRESHOLDS:
    print(f"t={t}: {len(dfs_by_t[t])} rows")


t=0.25: 850 rows
t=0.5: 850 rows
t=0.75: 850 rows
t=0.9: 850 rows


In [6]:
metrics_rows = []

for t in THRESHOLDS:
    df_t = dfs_by_t[t]

    metrics_rows.append({
        "threshold": t,
        "accuracy_at_t": accuracy_at_threshold(df_t, t),
        "coverage": coverage(df_t, t),
        "penalty_mean": penalty_adjusted_mean(df_t, t),
        "overconf_rate": overconfidence_rate(df_t, t),
        "answered_n": int((df_t["confidence"] > t).sum()),
        "total_n": len(df_t)
    })

metrics_df = pd.DataFrame(metrics_rows).sort_values("threshold").reset_index(drop=True)
metrics_df

,threshold,accuracy_at_t,coverage,penalty_mean,overconf_rate,answered_n,total_n
0,0.25,7.97,97.41,-24.70,90,828,850
1,0.50,9.03,97.76,-73.96,89,831,850
2,0.75,7.13,97.29,-226.38,90,827,850
3,0.90,8.22,97.29,-667.16,89,827,850


In [7]:
eval_table = (
    metrics_df[["threshold", "accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]]
    .set_index("threshold")
    .sort_index()
)

print("4×4 Evaluation Table:")
display(eval_table)

eval_table.to_csv(OUTPUT_PATH / "qwen-mmlu-metric-eval.csv")

print(f"Saved results in: {OUTPUT_PATH}")

4×4 Evaluation Table:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,7.97,97.41,-24.70,90
0.50,9.03,97.76,-73.96,89
0.75,7.13,97.29,-226.38,90
0.90,8.22,97.29,-667.16,89


Saved results in: outputs


### Baseline Evaluation

In [8]:
coverage_floor = 0.3

eligible = eval_table[eval_table["coverage"] >= coverage_floor]
print("DEBUG — eligible rows:")
display(eligible)

best_row = eligible.loc[eligible["accuracy_at_t"].idxmax()]
t_star = best_row.name

print(f"Selected t* = {t_star}")
display(best_row)

DEBUG — eligible rows:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,7.97,97.41,-24.70,90
0.50,9.03,97.76,-73.96,89
0.75,7.13,97.29,-226.38,90
0.90,8.22,97.29,-667.16,89


Selected t* = 0.5


accuracy_at_t     9.03
coverage         97.76
penalty_mean    -73.96
overconf_rate    89.00
Name: 0.5, dtype: float64

In [9]:
import numpy as np

df = pd.read_csv(CSV_PATH)

binary_acc = np.mean(df["predicted_answer"] == df["answer"])

wrong_rate = 1 - binary_acc

binary_row = {
    "accuracy_at_t": round(100*binary_acc,2),
    "coverage": 1.0,
    "penalty_mean": np.nan,
    "overconf_rate": round(100*wrong_rate,2),
}

abstain_row = {
    "accuracy_at_t": 0.0,
    "coverage": 0.0,
    "penalty_mean": 0.0,
    "overconf_rate": 0.0,
}

print("Binary-grading baseline:")
display(pd.DataFrame([binary_row], index=["Binary"]))
print("Always-abstain baseline:")
display(pd.DataFrame([abstain_row], index=["Abstain"]))


Binary-grading baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Binary,7.88,1.0,NaN,92.12


Always-abstain baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Abstain,0.0,0.0,0.0,0.0


In [10]:
headline_df = pd.DataFrame([
    best_row[["accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]],
    pd.Series(binary_row),
    pd.Series(abstain_row)
], index=[f"Confidence-aware (t*={t_star})", "Binary grading", "Always abstain"])

print("=== 3x4 Headline Evaluation Table ===")
display(headline_df)
headline_df.to_csv(OUTPUT_PATH / "qwen-mmlu-baseline-eval.csv")


=== 3x4 Headline Evaluation Table ===


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Confidence-aware (t*=0.5),9.03,97.76,-73.96,89.00
Binary grading,7.88,1.00,NaN,92.12
Always abstain,0.00,0.00,0.00,0.00


## GPT Evaluation

In [11]:
CSV_PATH = Path("../inference/outputs/gpt-mmlu.csv")
OUTPUT_PATH = Path("outputs")


In [12]:
df = pd.read_csv(CSV_PATH)

df["answer"] = df["answer"].astype(str).str.strip().str.upper()
df["predicted_answer"] = df["predicted_answer"].astype(str).str.strip().str.upper()
df["confidence"] = round(100*pd.to_numeric(df["confidence"], errors="coerce").fillna(0.0),2)
df["threshold"] = pd.to_numeric(df["threshold"], errors="coerce")

print(f"Loaded {len(df)} rows from {CSV_PATH.name}")
print("Thresholds found:", sorted(df["threshold"].unique()))
df.head(3)

Loaded 3400 rows from gpt-mmlu.csv
Thresholds found: [np.float64(0.25), np.float64(0.5), np.float64(0.75), np.float64(0.9)]


,id,threshold,question,choices,answer,predicted_answer,confidence
0,596,0.25,A manufacturer is currently selling 2000 units...,['$2.50' '$1.90' '$2.70' '$2.60' '$1.80' '$2.2...,F,D,33.33
1,814,0.25,Fred Lowes is a typewriter salesman. He receiv...,['$210' '$200' '$225' '$175' '$195' '$150' '$2...,I,B,66.67
2,817,0.25,"Mary Redmond purchased a $28,500 home with 20%...",['$305' '$190' '$171' '$285.50' '$399' '$323' ...,F,C,50.00


In [13]:
dfs_by_t = {t: df[df["threshold"] == t].copy() for t in THRESHOLDS}

for t in THRESHOLDS:
    print(f"t={t}: {len(dfs_by_t[t])} rows")


t=0.25: 850 rows
t=0.5: 850 rows
t=0.75: 850 rows
t=0.9: 850 rows


In [14]:
metrics_rows = []

for t in THRESHOLDS:
    df_t = dfs_by_t[t]

    metrics_rows.append({
        "threshold": t,
        "accuracy_at_t": accuracy_at_threshold(df_t, t),
        "coverage": coverage(df_t, t),
        "penalty_mean": penalty_adjusted_mean(df_t, t),
        "overconf_rate": overconfidence_rate(df_t, t),
        "answered_n": int((df_t["confidence"] > t).sum()),
        "total_n": len(df_t)
    })

metrics_df = pd.DataFrame(metrics_rows).sort_values("threshold").reset_index(drop=True)
metrics_df


,threshold,accuracy_at_t,coverage,penalty_mean,overconf_rate,answered_n,total_n
0,0.25,19.76,100.00,-20.72,80,850,850
1,0.50,19.18,100.00,-63.26,81,850,850
2,0.75,20.39,72.12,-134.84,57,613,850
3,0.90,0.00,0.00,0.00,0,0,850


In [15]:
eval_table = (
    metrics_df[["threshold", "accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]]
    .set_index("threshold")
    .sort_index()
)

print("4×4 Evaluation Table:")
display(eval_table)

eval_table.to_csv(OUTPUT_PATH / "gpt-mmlu-metric-eval.csv")

print(f"Saved results in: {OUTPUT_PATH}")


4×4 Evaluation Table:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,19.76,100.00,-20.72,80
0.50,19.18,100.00,-63.26,81
0.75,20.39,72.12,-134.84,57
0.90,0.00,0.00,0.00,0


Saved results in: outputs


### Baseline Evaluation

In [16]:
coverage_floor = 0.3

eligible = eval_table[eval_table["coverage"] >= coverage_floor]
print("DEBUG — eligible rows:")
display(eligible)

best_row = eligible.loc[eligible["accuracy_at_t"].idxmax()]
t_star = best_row.name

print(f"Selected t* = {t_star}")
display(best_row)

DEBUG — eligible rows:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,19.76,100.00,-20.72,80
0.50,19.18,100.00,-63.26,81
0.75,20.39,72.12,-134.84,57


Selected t* = 0.75


accuracy_at_t     20.39
coverage          72.12
penalty_mean    -134.84
overconf_rate     57.00
Name: 0.75, dtype: float64

In [17]:
import numpy as np

df = pd.read_csv(CSV_PATH)

binary_acc = np.mean(df["predicted_answer"] == df["answer"])

wrong_rate = 1 - binary_acc

binary_row = {
    "accuracy_at_t": round(100*binary_acc,2),
    "coverage": 1.0,
    "penalty_mean": np.nan, 
    "overconf_rate": round(100*wrong_rate,2),
}

abstain_row = {
    "accuracy_at_t": 0.0,
    "coverage": 0.0,
    "penalty_mean": 0.0,
    "overconf_rate": 0.0,
}

print("Binary-grading baseline:")
display(pd.DataFrame([binary_row], index=["Binary"]))
print("Always-abstain baseline:")
display(pd.DataFrame([abstain_row], index=["Abstain"]))


Binary-grading baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Binary,13.41,1.0,NaN,86.59


Always-abstain baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Abstain,0.0,0.0,0.0,0.0


In [18]:
headline_df = pd.DataFrame([
    best_row[["accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]],
    pd.Series(binary_row),
    pd.Series(abstain_row)
], index=[f"Confidence-aware (t*={t_star})", "Binary grading", "Always abstain"])

print("=== 3x4 Headline Evaluation Table ===")
display(headline_df)
headline_df.to_csv(OUTPUT_PATH / "gpt-mmlu-baseline-eval.csv")

=== 3x4 Headline Evaluation Table ===


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Confidence-aware (t*=0.75),20.39,72.12,-134.84,57.00
Binary grading,13.41,1.00,NaN,86.59
Always abstain,0.00,0.00,0.00,0.00


## Claude Evaluation

In [19]:
CSV_PATH = Path("../inference/outputs/claude-mmlu.csv")
OUTPUT_PATH = Path("outputs")


In [20]:
df = pd.read_csv(CSV_PATH)

df["answer"] = df["answer"].astype(str).str.strip().str.upper()
df["predicted_answer"] = df["predicted_answer"].astype(str).str.strip().str.upper()
df["confidence"] = round(100*pd.to_numeric(df["confidence"], errors="coerce").fillna(0.0),2)
df["threshold"] = pd.to_numeric(df["threshold"], errors="coerce")

print(f" Loaded {len(df)} rows from {CSV_PATH.name}")
print("Thresholds found:", sorted(df["threshold"].unique()))
df.head(3)

 Loaded 3400 rows from claude-mmlu.csv
Thresholds found: [np.float64(0.25), np.float64(0.5), np.float64(0.75), np.float64(0.9)]


,id,threshold,question,choices,answer,predicted_answer,confidence
0,596,0.25,A manufacturer is currently selling 2000 units...,['$2.50' '$1.90' '$2.70' '$2.60' '$1.80' '$2.2...,F,I,100.0
1,814,0.25,Fred Lowes is a typewriter salesman. He receiv...,['$210' '$200' '$225' '$175' '$195' '$150' '$2...,I,I,100.0
2,817,0.25,"Mary Redmond purchased a $28,500 home with 20%...",['$305' '$190' '$171' '$285.50' '$399' '$323' ...,F,I,100.0


In [21]:
dfs_by_t = {t: df[df["threshold"] == t].copy() for t in THRESHOLDS}

for t in THRESHOLDS:
    print(f"t={t}: {len(dfs_by_t[t])} rows")

t=0.25: 850 rows
t=0.5: 850 rows
t=0.75: 850 rows
t=0.9: 850 rows


In [22]:
metrics_rows = []

for t in THRESHOLDS:
    df_t = dfs_by_t[t]

    metrics_rows.append({
        "threshold": t,
        "accuracy_at_t": accuracy_at_threshold(df_t, t),
        "coverage": coverage(df_t, t),
        "penalty_mean": penalty_adjusted_mean(df_t, t),
        "overconf_rate": overconfidence_rate(df_t, t),
        "answered_n": int((df_t["confidence"] > t).sum()),
        "total_n": len(df_t)
    })

metrics_df = pd.DataFrame(metrics_rows).sort_values("threshold").reset_index(drop=True)

In [23]:
eval_table = (
    metrics_df[["threshold", "accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]]
    .set_index("threshold")
    .sort_index()
)

print("4×4 Evaluation Table:")
display(eval_table)

eval_table.to_csv(OUTPUT_PATH / "claude-mmlu-metric-eval.csv")

print(f"Saved results in: {OUTPUT_PATH}")


4×4 Evaluation Table:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,13.45,98.00,-26.35,85
0.50,13.82,97.88,-78.79,84
0.75,12.84,98.94,-246.70,86
0.90,11.76,99.06,-757.54,87


Saved results in: outputs


### Baseline Evaluation

In [24]:
coverage_floor = 0.3

eligible = eval_table[eval_table["coverage"] >= coverage_floor]
print("DEBUG — eligible rows:")
display(eligible)

best_row = eligible.loc[eligible["accuracy_at_t"].idxmax()]
t_star = best_row.name

print(f"Selected t* = {t_star}")
display(best_row)

DEBUG — eligible rows:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,13.45,98.00,-26.35,85
0.50,13.82,97.88,-78.79,84
0.75,12.84,98.94,-246.70,86
0.90,11.76,99.06,-757.54,87


Selected t* = 0.5


accuracy_at_t    13.82
coverage         97.88
penalty_mean    -78.79
overconf_rate    84.00
Name: 0.5, dtype: float64

In [25]:
import numpy as np

df = pd.read_csv(CSV_PATH)
binary_acc = np.mean(df["predicted_answer"] == df["answer"])

wrong_rate = 1 - binary_acc

binary_row = {
    "accuracy_at_t": round(100*binary_acc,2),
    "coverage": 1.0,
    "penalty_mean": np.nan,
    "overconf_rate": round(100*wrong_rate,2),
}

abstain_row = {
    "accuracy_at_t": 0.0,
    "coverage": 0.0,
    "penalty_mean": 0.0,
    "overconf_rate": 0.0,
}

print("Binary-grading baseline:")
display(pd.DataFrame([binary_row], index=["Binary"]))
print("Always-abstain baseline:")
display(pd.DataFrame([abstain_row], index=["Abstain"]))


Binary-grading baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Binary,12.76,1.0,NaN,87.24


Always-abstain baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Abstain,0.0,0.0,0.0,0.0


In [26]:
headline_df = pd.DataFrame([
    best_row[["accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]],
    pd.Series(binary_row),
    pd.Series(abstain_row)
], index=[f"Confidence-aware (t*={t_star})", "Binary grading", "Always abstain"])

print("=== 3x4 Headline Evaluation Table ===")
display(headline_df)
headline_df.to_csv(OUTPUT_PATH / "claude-mmlu-baseline-eval.csv")


=== 3x4 Headline Evaluation Table ===


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Confidence-aware (t*=0.5),13.82,97.88,-78.79,84.00
Binary grading,12.76,1.00,NaN,87.24
Always abstain,0.00,0.00,0.00,0.00


## LLama Evaluation

In [27]:
CSV_PATH = Path("../inference/outputs/llama-mmlu.csv")
OUTPUT_PATH = Path("outputs")


In [28]:
df = pd.read_csv(CSV_PATH)

df["answer"] = df["answer"].astype(str).str.strip().str.upper()
df["predicted_answer"] = df["predicted_answer"].astype(str).str.strip().str.upper()
df["confidence"] = round(100*pd.to_numeric(df["confidence"], errors="coerce").fillna(0.0),2)
df["threshold"] = pd.to_numeric(df["threshold"], errors="coerce")

print(f"Loaded {len(df)} rows from {CSV_PATH.name}")
print("Thresholds found:", sorted(df["threshold"].unique()))
df.head(3)

Loaded 3400 rows from llama-mmlu.csv
Thresholds found: [np.float64(0.25), np.float64(0.5), np.float64(0.75), np.float64(0.9)]


,id,threshold,question,choices,answer,predicted_answer,confidence
0,596,0.25,A manufacturer is currently selling 2000 units...,['$2.50' '$1.90' '$2.70' '$2.60' '$1.80' '$2.2...,F,E,50.00
1,814,0.25,Fred Lowes is a typewriter salesman. He receiv...,['$210' '$200' '$225' '$175' '$195' '$150' '$2...,I,E,33.33
2,817,0.25,"Mary Redmond purchased a $28,500 home with 20%...",['$305' '$190' '$171' '$285.50' '$399' '$323' ...,F,A,66.67


In [29]:
dfs_by_t = {t: df[df["threshold"] == t].copy() for t in THRESHOLDS}

for t in THRESHOLDS:
    print(f"t={t}: {len(dfs_by_t[t])} rows")

t=0.25: 850 rows
t=0.5: 850 rows
t=0.75: 850 rows
t=0.9: 850 rows


In [30]:
metrics_rows = []

for t in THRESHOLDS:
    df_t = dfs_by_t[t]

    metrics_rows.append({
        "threshold": t,
        "accuracy_at_t": accuracy_at_threshold(df_t, t),
        "coverage": coverage(df_t, t),
        "penalty_mean": penalty_adjusted_mean(df_t, t),
        "overconf_rate": overconfidence_rate(df_t, t),
        "answered_n": int((df_t["confidence"] > t).sum()),
        "total_n": len(df_t)
    })

metrics_df = pd.DataFrame(metrics_rows).sort_values("threshold").reset_index(drop=True)


In [31]:
eval_table = (
    metrics_df[["threshold", "accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]]
    .set_index("threshold")
    .sort_index()
)

print("4×4 Evaluation Table:")
display(eval_table)

eval_table.to_csv(OUTPUT_PATH / "llama-mmlu-metric-eval.csv")

print(f"Saved results in: {OUTPUT_PATH}")


4×4 Evaluation Table:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,9.33,89.53,-20.62,81
0.50,7.01,87.29,-59.87,81
0.75,9.90,90.35,-184.05,81
0.90,6.76,88.71,-546.70,83


Saved results in: outputs


### Baseline

In [32]:
coverage_floor = 0.3

eligible = eval_table[eval_table["coverage"] >= coverage_floor]
print("DEBUG — eligible rows:")
display(eligible)

best_row = eligible.loc[eligible["accuracy_at_t"].idxmax()]
t_star = best_row.name

print(f"Selected t* = {t_star}")
display(best_row)

DEBUG — eligible rows:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,9.33,89.53,-20.62,81
0.50,7.01,87.29,-59.87,81
0.75,9.90,90.35,-184.05,81
0.90,6.76,88.71,-546.70,83


Selected t* = 0.75


accuracy_at_t      9.90
coverage          90.35
penalty_mean    -184.05
overconf_rate     81.00
Name: 0.75, dtype: float64

In [33]:
import numpy as np

df = pd.read_csv(CSV_PATH)

binary_acc = np.mean(df["predicted_answer"] == df["answer"])

wrong_rate = 1 - binary_acc

binary_row = {
    "accuracy_at_t": round(100*binary_acc,2),
    "coverage": 1.0,
    "penalty_mean": np.nan,
    "overconf_rate": round(100*wrong_rate,2),
}

abstain_row = {
    "accuracy_at_t": 0.0,
    "coverage": 0.0,
    "penalty_mean": 0.0,
    "overconf_rate": 0.0,
}

print("Binary-grading baseline:")
display(pd.DataFrame([binary_row], index=["Binary"]))
print("Always-abstain baseline:")
display(pd.DataFrame([abstain_row], index=["Abstain"]))


Binary-grading baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Binary,7.35,1.0,NaN,92.65


Always-abstain baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Abstain,0.0,0.0,0.0,0.0


In [34]:
headline_df = pd.DataFrame([
    best_row[["accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]],
    pd.Series(binary_row),
    pd.Series(abstain_row)
], index=[f"Confidence-aware (t*={t_star})", "Binary grading", "Always abstain"])

print("=== 3x4 Headline Evaluation Table ===")
display(headline_df)
headline_df.to_csv(OUTPUT_PATH / "llama-mmlu-baseline-eval.csv")


=== 3x4 Headline Evaluation Table ===


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Confidence-aware (t*=0.75),9.90,90.35,-184.05,81.00
Binary grading,7.35,1.00,NaN,92.65
Always abstain,0.00,0.00,0.00,0.00


## Gemini Evaluation

In [35]:
CSV_PATH = Path("../inference/outputs/gemini-mmlu.csv")
OUTPUT_PATH = Path("outputs")


In [36]:
df = pd.read_csv(CSV_PATH)

df["answer"] = df["answer"].astype(str).str.strip().str.upper()
df["predicted_answer"] = df["predicted_answer"].astype(str).str.strip().str.upper()
df["confidence"] = round(100*pd.to_numeric(df["confidence"], errors="coerce").fillna(0.0),2)
df["threshold"] = pd.to_numeric(df["threshold"], errors="coerce")

print(f"Loaded {len(df)} rows from {CSV_PATH.name}")
print("Thresholds found:", sorted(df["threshold"].unique()))
df.head(3)

Loaded 3400 rows from gemini-mmlu.csv
Thresholds found: [np.float64(0.25), np.float64(0.5), np.float64(0.75), np.float64(0.9)]


,id,threshold,question,choices,answer,predicted_answer,confidence
0,596,0.25,A manufacturer is currently selling 2000 units...,['$2.50' '$1.90' '$2.70' '$2.60' '$1.80' '$2.2...,F,X,100.0
1,814,0.25,Fred Lowes is a typewriter salesman. He receiv...,['$210' '$200' '$225' '$175' '$195' '$150' '$2...,I,S,100.0
2,817,0.25,"Mary Redmond purchased a $28,500 home with 20%...",['$305' '$190' '$171' '$285.50' '$399' '$323' ...,F,S,100.0


In [37]:
dfs_by_t = {t: df[df["threshold"] == t].copy() for t in THRESHOLDS}

for t in THRESHOLDS:
    print(f"t={t}: {len(dfs_by_t[t])} rows")


t=0.25: 850 rows
t=0.5: 850 rows
t=0.75: 850 rows
t=0.9: 850 rows


In [38]:
metrics_rows = []

for t in THRESHOLDS:
    df_t = dfs_by_t[t]

    metrics_rows.append({
        "threshold": t,
        "accuracy_at_t": accuracy_at_threshold(df_t, t),
        "coverage": coverage(df_t, t),
        "penalty_mean": penalty_adjusted_mean(df_t, t),
        "overconf_rate": overconfidence_rate(df_t, t),
        "answered_n": int((df_t["confidence"] > t).sum()),
        "total_n": len(df_t)
    })

metrics_df = pd.DataFrame(metrics_rows).sort_values("threshold").reset_index(drop=True)
metrics_df


,threshold,accuracy_at_t,coverage,penalty_mean,overconf_rate,answered_n,total_n
0,0.25,12.35,67.65,-17.01,59,575,850
1,0.50,12.73,90.59,-67.47,79,770,850
2,0.75,11.95,90.59,-205.99,80,770,850
3,0.90,12.73,90.59,-605.76,79,770,850


In [39]:
eval_table = (
    metrics_df[["threshold", "accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]]
    .set_index("threshold")
    .sort_index()
)

print("4×4 Evaluation Table:")
display(eval_table)

eval_table.to_csv(OUTPUT_PATH / "gemini-mmlu-metric-eval.csv")

print(f"Saved results in: {OUTPUT_PATH}")


4×4 Evaluation Table:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,12.35,67.65,-17.01,59
0.50,12.73,90.59,-67.47,79
0.75,11.95,90.59,-205.99,80
0.90,12.73,90.59,-605.76,79


Saved results in: outputs


### Baseline Evaluation

In [40]:
coverage_floor = 0.3

eligible = eval_table[eval_table["coverage"] >= coverage_floor]
print("DEBUG — eligible rows:")
display(eligible)

best_row = eligible.loc[eligible["accuracy_at_t"].idxmax()]
t_star = best_row.name

print(f"Selected t* = {t_star}")
display(best_row)

DEBUG — eligible rows:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,12.35,67.65,-17.01,59
0.50,12.73,90.59,-67.47,79
0.75,11.95,90.59,-205.99,80
0.90,12.73,90.59,-605.76,79


Selected t* = 0.5


accuracy_at_t    12.73
coverage         90.59
penalty_mean    -67.47
overconf_rate    79.00
Name: 0.5, dtype: float64

In [41]:
import numpy as np

df = pd.read_csv(CSV_PATH)

binary_acc = np.mean(df["predicted_answer"] == df["answer"])

wrong_rate = 1 - binary_acc

binary_row = {
    "accuracy_at_t": round(100*binary_acc,2),
    "coverage": 1.0,
    "penalty_mean": np.nan,
    "overconf_rate": round(100*wrong_rate,2),
}

abstain_row = {
    "accuracy_at_t": 0.0,
    "coverage": 0.0,
    "penalty_mean": 0.0,
    "overconf_rate": 0.0,
}

print("Binary-grading baseline:")
display(pd.DataFrame([binary_row], index=["Binary"]))
print("Always-abstain baseline:")
display(pd.DataFrame([abstain_row], index=["Abstain"]))


Binary-grading baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Binary,10.56,1.0,NaN,89.44


Always-abstain baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Abstain,0.0,0.0,0.0,0.0


In [42]:
headline_df = pd.DataFrame([
    best_row[["accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]],
    pd.Series(binary_row),
    pd.Series(abstain_row)
], index=[f"Confidence-aware (t*={t_star})", "Binary grading", "Always abstain"])

print("=== 3x4 Headline Evaluation Table ===")
display(headline_df)
headline_df.to_csv(OUTPUT_PATH / "gemini-mmlu-baseline-eval.csv")


=== 3x4 Headline Evaluation Table ===


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Confidence-aware (t*=0.5),12.73,90.59,-67.47,79.00
Binary grading,10.56,1.00,NaN,89.44
Always abstain,0.00,0.00,0.00,0.00


## Mistral Evaluation

In [43]:
CSV_PATH = Path("../inference/outputs/mistral-mmlu.csv")
OUTPUT_PATH = Path("outputs")


In [44]:
df = pd.read_csv(CSV_PATH)

df["answer"] = df["answer"].astype(str).str.strip().str.upper()
df["predicted_answer"] = df["predicted_answer"].astype(str).str.strip().str.upper()
df["confidence"] = round(100*pd.to_numeric(df["confidence"], errors="coerce").fillna(0.0),2)
df["threshold"] = pd.to_numeric(df["threshold"], errors="coerce")

print(f" Loaded {len(df)} rows from {CSV_PATH.name}")
print("Thresholds found:", sorted(df["threshold"].unique()))
df.head(3)

 Loaded 3400 rows from mistral-mmlu.csv
Thresholds found: [np.float64(0.25), np.float64(0.5), np.float64(0.75), np.float64(0.9)]


,id,threshold,question,choices,answer,predicted_answer,confidence
0,596,0.25,A manufacturer is currently selling 2000 units...,['$2.50' '$1.90' '$2.70' '$2.60' '$1.80' '$2.2...,F,IDK,100.00
1,814,0.25,Fred Lowes is a typewriter salesman. He receiv...,['$210' '$200' '$225' '$175' '$195' '$150' '$2...,I,A,100.00
2,817,0.25,"Mary Redmond purchased a $28,500 home with 20%...",['$305' '$190' '$171' '$285.50' '$399' '$323' ...,F,IDK,83.33


In [45]:
dfs_by_t = {t: df[df["threshold"] == t].copy() for t in THRESHOLDS}

for t in THRESHOLDS:
    print(f"t={t}: {len(dfs_by_t[t])} rows")


t=0.25: 850 rows
t=0.5: 850 rows
t=0.75: 850 rows
t=0.9: 850 rows


In [46]:
metrics_rows = []

for t in THRESHOLDS:
    df_t = dfs_by_t[t]

    metrics_rows.append({
        "threshold": t,
        "accuracy_at_t": accuracy_at_threshold(df_t, t),
        "coverage": coverage(df_t, t),
        "penalty_mean": penalty_adjusted_mean(df_t, t),
        "overconf_rate": overconfidence_rate(df_t, t),
        "answered_n": int((df_t["confidence"] > t).sum()),
        "total_n": len(df_t)
    })

metrics_df = pd.DataFrame(metrics_rows).sort_values("threshold").reset_index(drop=True)
metrics_df

,threshold,accuracy_at_t,coverage,penalty_mean,overconf_rate,answered_n,total_n
0,0.25,8.99,98.12,-24.44,89,834,850
1,0.50,8.86,98.24,-74.80,90,835,850
2,0.75,8.86,98.24,-222.95,90,835,850
3,0.90,8.82,98.71,-682.45,90,839,850


In [47]:
eval_table = (
    metrics_df[["threshold", "accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]]
    .set_index("threshold")
    .sort_index()
)

print("4×4 Evaluation Table:")
display(eval_table)

eval_table.to_csv(OUTPUT_PATH / "mistral-mmlu-metric-eval.csv")

print(f"Saved results in: {OUTPUT_PATH}")


4×4 Evaluation Table:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,8.99,98.12,-24.44,89
0.50,8.86,98.24,-74.80,90
0.75,8.86,98.24,-222.95,90
0.90,8.82,98.71,-682.45,90


Saved results in: outputs


### Baseline

In [48]:
coverage_floor = 0.3

eligible = eval_table[eval_table["coverage"] >= coverage_floor]
print("DEBUG — eligible rows:")
display(eligible)

best_row = eligible.loc[eligible["accuracy_at_t"].idxmax()]
t_star = best_row.name

print(f"Selected t* = {t_star}")
display(best_row)

DEBUG — eligible rows:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
threshold,,,,
0.25,8.99,98.12,-24.44,89
0.50,8.86,98.24,-74.80,90
0.75,8.86,98.24,-222.95,90
0.90,8.82,98.71,-682.45,90


Selected t* = 0.25


accuracy_at_t     8.99
coverage         98.12
penalty_mean    -24.44
overconf_rate    89.00
Name: 0.25, dtype: float64

In [49]:
import numpy as np
df = pd.read_csv(CSV_PATH)

binary_acc = np.mean(df["predicted_answer"] == df["answer"])

wrong_rate = 1 - binary_acc

binary_row = {
    "accuracy_at_t": round(100*binary_acc,2),
    "coverage": 1.0,
    "penalty_mean": np.nan,
    "overconf_rate": round(100*wrong_rate,2),
}

abstain_row = {
    "accuracy_at_t": 0.0,
    "coverage": 0.0,
    "penalty_mean": 0.0,
    "overconf_rate": 0.0,
}

print("Binary-grading baseline:")
display(pd.DataFrame([binary_row], index=["Binary"]))
print("Always-abstain baseline:")
display(pd.DataFrame([abstain_row], index=["Abstain"]))


Binary-grading baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Binary,8.74,1.0,NaN,91.26


Always-abstain baseline:


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Abstain,0.0,0.0,0.0,0.0


In [50]:
headline_df = pd.DataFrame([
    best_row[["accuracy_at_t", "coverage", "penalty_mean", "overconf_rate"]],
    pd.Series(binary_row),
    pd.Series(abstain_row)
], index=[f"Confidence-aware (t*={t_star})", "Binary grading", "Always abstain"])

print("=== 3x4 Headline Evaluation Table ===")
display(headline_df)
headline_df.to_csv(OUTPUT_PATH / "mistral-mmlu-baseline-eval.csv")

=== 3x4 Headline Evaluation Table ===


,accuracy_at_t,coverage,penalty_mean,overconf_rate
Confidence-aware (t*=0.25),8.99,98.12,-24.44,89.00
Binary grading,8.74,1.00,NaN,91.26
Always abstain,0.00,0.00,0.00,0.00
